In [4]:
from keras.models import model_from_json
from neural_net import *

def warmstart(test_traj: int = 1, load_model:bool = True):
    if load_model:
        json_file = open('model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        neuralNet = model_from_json(loaded_model_json)
        loaded_model.load_weights("model.h5")
        print("Loaded model from disk")

        neuralNet.compile(loss='mean_squared_error',
                          optimizer=rms,
                          metrics=['mean_squared_error', "mean_absolute_error"])
        
    else:
        neuralNet = train_net(ntraj = 10000)
        neuralNet.compile(loss='mean_squared_error',
                          optimizer=rms,
                          metrics=['mean_squared_error', "mean_absolute_error"])
    for _ in range(test_traj):
        
        model = cartpole_model()

        x0 = [ random.uniform(0,1), random.uniform(-3.14, 3.14), random.uniform(0., 0.99)
                        , random.uniform(0., .99)]
        
        prediction = neuralNet.predict(x0)
        
        #results = 

        T  = 100
        problem = crocoddyl.ShootingProblem(np.matrix(x0).T, [ model ]*T, model)
        ddp = crocoddyl.SolverDDP(problem)
        # Solving this problem
        done = ddp.solve([], [], 1000)
        del ddp.xs[0]
        
        
        

In [ ]:
from keras.models import model_from_json
from neural_net import *
import numpy as np
import random
from keras import optimizers

rms = optimizers.RMSprop(learning_rate=0.001, rho=0.9)

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
neuralNet = model_from_json(loaded_model_json)
neuralNet.load_weights("model.h5")
print("Loaded model from disk")

neuralNet.compile(loss='mean_squared_error',
                  optimizer=rms,
                  metrics=['mean_squared_error', "mean_absolute_error"])

In [49]:
model = cartpole_model()
model2 = cartpole_model()

x0 = [ random.uniform(0,1), random.uniform(-3.14, 3.14), random.uniform(0., 0.99)
                , random.uniform(0., .99)]

#print(np.array(x0).reshape(1, 4))

prediction = neuralNet.predict(np.array(x0).reshape(1,4))
prediction = prediction.reshape(100, 5)
##Every fifth element is control, the first 4 elements are state
ddp_xs, ddp_us = [], []
ddp_xs.append(np.matrix(x0).T)
xs = prediction[:,0:4]
us = prediction[:,4]

for _ in range(xs.shape[0]):
    ddp_xs.append(np.matrix(xs[_]).T)

for _ in  range(us.shape[0]):
    ddp_us.append(np.matrix(us[_]).T)




T  = 100
problem = crocoddyl.ShootingProblem(np.matrix(x0).T, [ model ]*T, model)
problem2 = crocoddyl.ShootingProblem(np.matrix(x0).T, [ model2 ]*T, model2)
ddp2 = crocoddyl.SolverDDP(problem2)
ddp = crocoddyl.SolverDDP(problem)
# Solving this problem
done = ddp.solve(ddp_xs, ddp_us, 1000)
done = ddp2.solve([], [], 1000)
print(ddp.iter)
print(ddp2.iter)

1000
295


In [52]:
%%capture
%matplotlib inline
from cartpole_utils import animateCartpole
from IPython.display import HTML

anim = animateCartpole(ddp_xs)
# HTML(anim.to_jshtml())
HTML(anim.to_html5_video())

In [71]:
%%capture
%matplotlib inline

# Create animation
anim = animateCartpole(ddp2.xs)
anim2 = animateCartpole(ddp_xs)

In [74]:
HTML(anim.to_html5_video()) 


In [75]:
HTML(anim2.to_html5_video()) 
